### Universidad Nacional de Lujan - Bases de Datos Masivas (11088) - Cavasin Nicolas #143501
# TP05-01 - Árboles de decisión

### Ejercicio 5:
Un Banco de Portugal realizó una campaña de marketing en busca de clientes de plazos fijos basada en llamados telefónicos. Se provee el dataset real *(bank-full.csv)* con más 45000 instancias y el detalle *(bank-names.txt)* de los datos registrados de cada una de las personas contactadas por la entidad bancaria:
- Realice las tareas necesarias para poder procesar el dataset en Scikit-Learn.
- Luego, genere el árbol de decisión, y optimice los resultados, con el objetivo de explicar cuáles son las características más importantes que permiten identificar a una persona que accederá o no al plazo fijo. Documente los resultados.

In [1]:
!rm bank-full.csv
!rm bank-names.txt
!wget https://raw.githubusercontent.com/bdm-unlu/2020/master/TPs/TP05/TP0501/bank-full.csv
!wget https://raw.githubusercontent.com/bdm-unlu/2020/master/TPs/TP05/TP0501/bank-names.txt

--2020-11-12 20:14:56--  https://raw.githubusercontent.com/bdm-unlu/2020/master/TPs/TP05/TP0501/bank-full.csv
Loaded CA certificate &#39;/etc/ssl/certs/ca-certificates.crt&#39;
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.216.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.216.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4610348 (4.4M) [text/plain]
Saving to: &#39;bank-full.csv&#39;

bank-full.csv       100%[===================&gt;]   4.40M  3.45MB/s    in 1.3s    

2020-11-12 20:14:57 (3.45 MB/s) - &#39;bank-full.csv&#39; saved [4610348/4610348]

--2020-11-12 20:14:58--  https://raw.githubusercontent.com/bdm-unlu/2020/master/TPs/TP05/TP0501/bank-names.txt
Loaded CA certificate &#39;/etc/ssl/certs/ca-certificates.crt&#39;
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.216.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.216.133|:443..

In [2]:
import pandas as pd

banco = pd.read_csv("bank-full.csv")

print(type(banco))
print(f'Cantidad de tuplas: {banco.shape[0]}')
print(f'Cantidad de columnas: {banco.shape[1]}')

banco.head()


&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
Cantidad de tuplas: 45211
Cantidad de columnas: 1


,"age;""job"";""marital"";""education"";""default"";""balance"";""housing"";""loan"";""contact"";""day"";""month"";""duration"";""campaign"";""pdays"";""previous"";""poutcome"";""y"""
0,"58;""management"";""married"";""tertiary"";""no"";2143..."
1,"44;""technician"";""single"";""secondary"";""no"";29;""..."
2,"33;""entrepreneur"";""married"";""secondary"";""no"";2..."
3,"47;""blue-collar"";""married"";""unknown"";""no"";1506..."
4,"33;""unknown"";""single"";""unknown"";""no"";1;""no"";""n..."


In [3]:
# Leo el archivo txt
with open('bank-names.txt') as metadata:
    print(metadata.read())

Citation Request:
  This dataset is public available for research. The details are described in [Moro et al., 2011]. 
  Please include this citation if you plan to use this database:

  [Moro et al., 2011] S. Moro, R. Laureano and P. Cortez. Using Data Mining for Bank Direct Marketing: An Application of the CRISP-DM Methodology. 
  In P. Novais et al. (Eds.), Proceedings of the European Simulation and Modelling Conference - ESM&#39;2011, pp. 117-121, Guimarães, Portugal, October, 2011. EUROSIS.

  Available at: [pdf] http://hdl.handle.net/1822/14838
                [bib] http://www3.dsi.uminho.pt/pcortez/bib/2011-esm-1.txt

1. Title: Bank Marketing

2. Sources
   Created by: Paulo Cortez (Univ. Minho) and Sérgio Moro (ISCTE-IUL) @ 2012
   
3. Past Usage:

  The full dataset was described and analyzed in:

  S. Moro, R. Laureano and P. Cortez. Using Data Mining for Bank Direct Marketing: An Application of the CRISP-DM Methodology. 
  In P. Novais et al. (Eds.), Proceedings of the Europe

Se puede observar que el dataset se encuentra almacenado únicamente en el archivo *bank-full.csv* mientras que *bank-names.txt* funciona como una pseudo-metadata que describe: quiénes son sus creadores, políticas de uso, con qué fin se utilizó, qué es lo que representa cada columna e indica que **no posee datos faltantes**.


Al cargar el dataset en memoria usando un *DataFrame*, presenta un formato extraño que posee una única columna en vez de 17 tal como explica el archivo *bank-names.txt*. Lo que hay que modificar es la manera en que se intenta leer a definiendo el parámetro ``sep=";"``. 

A continuación se repite la lectura pero esta vez de manera correcta:

In [4]:
# Cambio el tipo de separador con el que se interpreta el archivo
banco = pd.read_csv("bank-full.csv", sep=";")

print(type(banco))
print(f'Cantidad de tuplas: {banco.shape[0]}')
print(f'Cantidad de columnas: {banco.shape[1]}')

banco.head()

&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
Cantidad de tuplas: 45211
Cantidad de columnas: 17


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


Se puede observar que el dataset posee varias columnas de texto que deben ser pre-procesadas para convertirlas en variables numéricas discretas y así poder ser utilizadas en la creación del árbol de clasificación. 

La transformación se hará sobre cada columna de manera individual y aplicando el método *LabelEncoder*. 

Adicionalmente, se eliminará el target (columna '*y*') del dataframe original.

In [5]:
# Importo el encoder
from sklearn.preprocessing import LabelEncoder

# Instancio el encoder
le = LabelEncoder()

# Defino el target
target_name = 'y'
target = banco.pop(target_name)

# Separo columnas numericas de columnas de texto
for col in banco.columns:
    if banco[col].dtype == 'object':
        banco[col] = le.fit_transform(banco[col])

# Defino las features ya transformadas
features_names = banco.columns
features = banco[features_names]

print(f'Target: {target_name}')
print(f'Features: {features_names}')

banco.head()

Target: y
Features: Index([&#39;age&#39;, &#39;job&#39;, &#39;marital&#39;, &#39;education&#39;, &#39;default&#39;, &#39;balance&#39;, &#39;housing&#39;,
       &#39;loan&#39;, &#39;contact&#39;, &#39;day&#39;, &#39;month&#39;, &#39;duration&#39;, &#39;campaign&#39;, &#39;pdays&#39;,
       &#39;previous&#39;, &#39;poutcome&#39;],
      dtype=&#39;object&#39;)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,58,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3
1,44,9,2,1,0,29,1,0,2,5,8,151,1,-1,0,3
2,33,2,1,1,0,2,1,1,2,5,8,76,1,-1,0,3
3,47,1,1,3,0,1506,1,0,2,5,8,92,1,-1,0,3
4,33,11,2,3,0,1,0,0,2,5,8,198,1,-1,0,3


Una vez que:
- Se transformaron las columnas necesarias.
- Se definió el target.
- Se definieron las features.

Se procede con la creación del arbol de clasificación:

In [11]:
# Importo la libreria que permite separar en training set y test set
from sklearn.model_selection import train_test_split as s

# Importo el arbol
from sklearn import tree

# Importo libreria para graficar
import graphviz

# Importo numpy
import numpy as np

# Importo las metricas para testear el modelo
from sklearn import metrics

# Divido el dataset en 80-20 para training y test respectivamente
features_train, features_test, target_train, target_test = s(features, target, random_state=0, test_size=0.4)

# Instancio el arbol por entropia
t = tree.DecisionTreeClassifier(criterion='entropy')

# Lo entreno
t.fit(features_train, target_train)

# Obtengo la prediccion con el set de prueba definido anteriormente
prediccion = t.predict(features_test)

# Convierto a array para recorrer
target_test = np.asarray(target_test)

# Muestro prediccion vs original
header = '{:<4}{:<2}{:<15}{:<2}{:<11}{}'.format('#', '|', 'Predicción', '|', 'Original','|')
print(header)
print('='*35, end='')

sep = '|'

for i in range(10):
    print(f"\n{str(i).ljust(4, ' ')}{sep.ljust(2, ' ')}{prediccion[i].ljust(15, ' ')}{sep.ljust(2, ' ')}{target_test[i].ljust(11, ' ')}{sep}\n", end='-'*35)

# Veo qué tan acertado estuvo
print(f'\n\nPrecisión del modelo: {metrics.accuracy_score(target_test, prediccion)}', end='\n\n')

# Muestro un reporte de clasificación con diferentes métricas sobre cada feature 
print(f'Reporte de clasificación: \n{metrics.classification_report(target_test, prediccion)}')


#   | Predicción     | Original   |
0   | no             | no         |
-----------------------------------
1   | yes            | yes        |
-----------------------------------
2   | no             | no         |
-----------------------------------
3   | no             | no         |
-----------------------------------
4   | no             | no         |
-----------------------------------
5   | no             | no         |
-----------------------------------
6   | no             | no         |
-----------------------------------
7   | no             | no         |
-----------------------------------
8   | no             | no         |
-----------------------------------
9   | no             | no         |
-----------------------------------

Precisión del modelo: 0.8730992535250207

Reporte de clasificación: 
              precision    recall  f1-score   support

          no       0.93      0.93      0.93     15957
         yes       0.46      0.47      0.46      2128

    accura

El modelo posee una precisión del 87.2% pero es un modelo con una cantidad de niveles importante ya que *graphviz* demora bastante en graficarlo. Es por esto que decidí no graficar esta versión cuyo peso en formato *.png* es ~8MB. 

___

A continuación se realiza una poda definiendo una profundidad máxima de 7 niveles y un mínimo de 35 samples por hoja y a esta versión parametrizada sí se la grafica:

In [20]:
# Instancio el arbol por entropia
t_param = tree.DecisionTreeClassifier(criterion='entropy', max_depth=7, min_samples_leaf=35)

# Lo entreno
t_param.fit(features_train, target_train)

# Almaceno el modelo en formato DOT
banco_dot = tree.export_graphviz(t_param
                    , out_file=None
                    , feature_names=features_names
                    , label='all'
                    , filled=True, rounded=True
                    , special_characters=True)

# Tambien lo guardo como PNG
graph = graphviz.Source(banco_dot)
graph.format = 'png'
graph.render('banco_param')

# Obtengo la prediccion con el set de prueba definido anteriormente
prediccion2 = t_param.predict(features_test)

# Convierto a array para recorrer
target_test = np.asarray(target_test)

# Muestro prediccion vs original
header = '{:<4}{:<2}{:<15}{:<2}{:<11}{}'.format('#', '|', 'Predicción', '|', 'Original','|')
print(header)
print('='*35, end='')

sep = '|'

for i in range(10):
    print(f"\n{str(i).ljust(4, ' ')}{sep.ljust(2, ' ')}{prediccion2[i].ljust(15, ' ')}{sep.ljust(2, ' ')}{target_test[i].ljust(11, ' ')}{sep}\n", end='-'*35)

# Veo qué tan acertado estuvo
print(f'\n\nPrecisión del modelo: {metrics.accuracy_score(target_test, prediccion2)}', end='\n\n')

# Muestro un reporte de clasificación con diferentes métricas sobre cada feature 
print(f'Reporte de clasificación: \n{metrics.classification_report(target_test, prediccion2)}')

#   | Predicción     | Original   |
0   | no             | no         |
-----------------------------------
1   | yes            | yes        |
-----------------------------------
2   | no             | no         |
-----------------------------------
3   | no             | no         |
-----------------------------------
4   | no             | no         |
-----------------------------------
5   | no             | no         |
-----------------------------------
6   | no             | no         |
-----------------------------------
7   | no             | no         |
-----------------------------------
8   | no             | no         |
-----------------------------------
9   | no             | no         |
-----------------------------------

Precisión del modelo: 0.8941111418302461

Reporte de clasificación: 
              precision    recall  f1-score   support

          no       0.92      0.96      0.94     15957
         yes       0.57      0.42      0.48      2128

    accura

Árbol parametrizado con 7 niveles máximos de profundidad y mínimo 35 samples por hoja:

![Arbol](banco_param.png)

## Conclusiones:

- Se puede observar que los features con mayor relevancia son:

    1. *Duration*: duración de la comunicación en milisegundos.

    2. *Contact*: forma de comunicación (teléfono, celular, etc.).

    3. *Balance*: promedio anual de ingresos en euros.

    4. *Month*: última vez que se contactaron con el cliente.

    5. *Housing*: posee un préstamo para construcción de hogar? (housing loan).
    
    6. *Pdays*: cantidad de días que pasaron desde la última vez que se contactaron con el cliente.
    

- Por lo detectado, es muy importante el diálogo con el cliente ya que la mayoría de las features con mayor nivel de *Ganancia de Información* dentro del dataset están relacionadas con este, o derivan de él. 
- Es decir, si el banco no insiste en entablar conversaciones es muy difícil que el cliente acceda al plazo fijo. 
- En este caso, para el modelo resultante las features de comunicación son más importantes que la cantidad de dinero que el cliente gane.